In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)     # 숫자가 높을수록 랜덤함.

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
prompt = template.format(country_a="Mexico", country_b="Thailand")
chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [5]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [6]:
# CommaOutputParser 테스트

p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [12]:
# 대답의 포맷을 지정하는 법 (구식방법)
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated a list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items = 10,
    #question = "What are the planets?"
    question = "What are the colors"
)

result = chat.predict_messages(prompt)

# 대답을 원하는 형식으로 만들기
p = CommaOutputParser()

p.parse(result.content)



['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [13]:
# 대답의 포맷을 지정하는 법 (최신 방법)
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated a list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}")
])

# 질문할 문자열 | 채팅모델 | 출력형식지정
# template | language model | output parser
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"Waht are the pokemons?"
})

['bulbasaur', 'charmander', 'squirtle', 'pikachu', 'jigglypuff']

In [14]:
# 변수 지정법
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(language="Greek", name="Socrates", country_a="Mexico", country_b="Thailand")

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')